<a href="https://colab.research.google.com/github/fcoliveira-utfpr/nasapower_validation/blob/main/validation_eto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Import libraries**
---

In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline
%config inlineBackend.figure_formats = ['svg']
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr